In [5]:
from konlpy.tag import Okt
import pandas as pd
import os
import numpy as np
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()

path_dir = 'data'

file_list = os.listdir(path_dir)

file_list=file_list[0:3000]
len(file_list)

3000

In [ ]:
from tqdm.notebook import tqdm
for file_name in tqdm(file_list):
    try:
        df=pd.read_csv(os.path.join(path_dir,file_name),error_bad_lines=False,header=None)
    except:
        continue
    df=df[[2,5]]
    df.columns=['rate','comment']
    
    #한글빼고 모두 제거
    df['comment'] = df['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    #nan  제거
    df = df.dropna(how = 'any')
    df=df.reset_index(drop=True)
    # 불용어
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    comments=[]
    for sentence in df['comment']:
        tmp = okt.morphs(sentence, stem=True) # 토큰화
        tmp = [word for word in tmp if not word in stopwords] # 불용어 제거
        comments.append(tmp)
    
    y=df['rate']

    #tokenizer 

    if (len(comments)!=y.shape[0]):
        continue
    # if x.shape!=y.shape:
    #     continue
    drop_comments = [index for index, sentence in enumerate(comments) if len(sentence) < 1]
    x = np.delete(comments, drop_comments, axis=0)
    y = y.drop(drop_comments, axis=0)
  

    if file_name==file_list[0]:
        x_train=x
        y_train=y
    else:
        if x.ndim!=y.ndim:
            continue
        x_train=np.concatenate((x_train,x),axis=0)
        y_train=np.concatenate((y_train,y),axis=0)

In [ ]:
dff=pd.DataFrame()
dff['x']=x_train
dff['y']=y_train

In [ ]:
dff.to_csv("data_0_3000.csv",index=False,encoding='utf-8-sig')

In [ ]:
import pandas as pd
a=pd.read_csv("data_0_3000.csv")
b=pd.read_csv("data_3000_6000.csv")
c=pd.read_csv("data_6000_12000.csv")
d=pd.read_csv("data_12000_14000.csv")
e=pd.read_csv("data_14000_all.csv")
vec=pd.concat([a,b,c,d,e])

In [ ]:
vec

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\test\anaconda3\envs\youtube\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1bccd1c4e65e>", line 1, in <module>
    vec
NameError: name 'vec' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\anaconda3\envs\youtube\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\anaconda3\envs\youtube\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\test\anaconda3\envs\youtube\l

In [ ]:
vec.to_csv("data_all.csv",index=False,encoding='utf-8-sig')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(vec['x'])

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
vocab_size = total_cnt - rare_cnt + 1
print(f"vocab_size={vocab_size}")

In [6]:
vec=pd.read_csv("data_all.csv")


ERROR! Session/line number was not unique in database. History logging moved to new session 151


In [7]:
vec

,x,y
0,"['배경음', '감동', '적', '인', '선율', '영사기', '에서', '나오...",10.0
1,"['불멸', '명작', '영화인', '을', '꿈꾸다', '사람', '라면', '반...",9.0
2,"['년', '훨씬', '지나다', '아직도', '생각', '가슴', '벅차다']",10.0
3,"['오랜', '만', '다시', '보아', '너무', '예쁘다', '사랑스럽다', ...",10.0
4,"['지금', '극장', '걸리다', '있다', '건', '축약', '판', '분',...",10.0
...,...,...
6482538,"['기', '대안', '고', '보다', '괜찮다']",8.0
6482539,"['기네스', '팰트로으', '연기', '너무', '인상', '적다', '영화', ...",10.0
6482540,"['기네스', '팰트로우', '만', '보이다', '영화', '노잼']",1.0
6482541,"['찌질하다', '순수하다', '살', '남자', '아이', '성장기', '부모',...",7.0


In [22]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(vec['x'])

In [34]:
tokenizer.texts_to_sequences(["'영화'"])

[[2]]

In [30]:
(tokenizer.word_index)

{'<OOV>': 1,
 "'영화'": 2,
 "'보다'": 3,
 "'을'": 4,
 "'재밌다'": 5,
 "'좋다'": 6,
 "'이다'": 7,
 "'너무'": 8,
 "'있다'": 9,
 "'정말'": 10,
 "'없다'": 11,
 "'만'": 12,
 "'다'": 13,
 "'진짜'": 14,
 "'적'": 15,
 "'같다'": 16,
 "'최고'": 17,
 "'감동'": 18,
 "'않다'": 19,
 "'연기'": 20,
 "'되다'": 21,
 "'재미있다'": 22,
 "'생각'": 23,
 "'로'": 24,
 "'아니다'": 25,
 "'점'": 26,
 "'요'": 27,
 "'에서'": 28,
 "'스토리'": 29,
 "'나오다'": 30,
 "'볼'": 31,
 "'것'": 32,
 "'ㅋㅋ'": 33,
 "'인'": 34,
 "'보고'": 35,
 "'더'": 36,
 "'그'": 37,
 "'배우'": 38,
 "'안'": 39,
 "'내'": 40,
 "'자다'": 41,
 "'재미'": 42,
 "'만들다'": 43,
 "'사람'": 44,
 "'평점'": 45,
 "'그냥'": 46,
 "'나'": 47,
 "'말'": 48,
 "'가다'": 49,
 "'고'": 50,
 "'지루하다'": 51,
 "'게'": 52,
 "'못'": 53,
 "'시간'": 54,
 "'내용'": 55,
 "'하고'": 56,
 "'중'": 57,
 "'마지막'": 58,
 "'ㅠㅠ'": 59,
 "'수'": 60,
 "'까지'": 61,
 "'때'": 62,
 "'네'": 63,
 "'들다'": 64,
 "'모르다'": 65,
 "'왜'": 66,
 "'역시'": 67,
 "'아'": 68,
 "'오다'": 69,
 "'그렇다'": 70,
 "'완전'": 71,
 "'주다'": 72,
 "'많다'": 73,
 "'꼭'": 74,
 "'이렇다'": 75,
 "'아깝다'": 76,
 "'넘다'": 77,
 "'ㅋ'": 78,
 "'괜찮다'

In [11]:
# tokenizer = Tokenizer(118798)
# tokenizer.fit_on_texts(vec['x'])
x_train = tokenizer.texts_to_sequences(vec['x'])

In [12]:
x_train

[[7884,
  17,
  14,
  33,
  3866,
  27344,
  27,
  29,
  2644,
  2851,
  93,
  3,
  2,
  6998,
  3066,
  3034,
  2976,
  1115,
  164,
  377,
  10],
 [7392, 168, 383, 3, 1796, 43, 392, 1975, 2, 178, 1],
 [209, 483, 854, 441, 22, 137, 2351],
 [184, 11, 95, 1411, 7, 335, 661, 802, 1],
 [164,
  303,
  1072,
  8,
  376,
  11866,
  751,
  139,
  11866,
  751,
  11,
  1764,
  139,
  73,
  2712,
  139,
  5611,
  156,
  2,
  370,
  237,
  685,
  3,
  106,
  59,
  8,
  46,
  296,
  14,
  2712,
  3,
  483,
  35,
  104,
  483,
  35,
  914,
  685,
  287],
 [67,
  207,
  460,
  334,
  1983,
  110,
  138,
  2920,
  20,
  93,
  72,
  344,
  18682,
  6982,
  12714,
  23,
  2920,
  319],
 [284,
  27,
  434,
  2,
  128,
  1402,
  6722,
  128,
  33,
  18682,
  6998,
  5798,
  154,
  3,
  118,
  12714,
  23,
  63,
  272,
  504,
  53,
  174,
  69,
  187,
  1424,
  349,
  1627,
  3214,
  2534,
  177,
  1,
  330,
  81,
  43,
  1605,
  3985,
  561,
  230,
  79],
 [423, 268, 31, 63, 58],
 [15397,
  27,
  867,
 

In [ ]:
drop_train = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
x = np.delete(x_train, drop_train, axis=0)
y = np.delete(np.array(vec['y']), drop_train, axis=0)
print(len(x))
print(len(y))

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in x))
print('리뷰의 평균 길이 :',sum(map(len, x))/len(y))
plt.hist([len(s) for s in x], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 40
below_threshold_len(max_len,x)

In [ ]:
x = pad_sequences(x, maxlen = max_len)

In [ ]:
new_data=pd.DataFrame()
new_data['comment']=x.tolist()
new_data['rate']=y
new_data.to_csv("data_0_all_finish.csv",index=False,encoding='utf-8-sig')

In [ ]:
new_data['rate'].value_counts()

In [ ]:
idx=new_data[new_data['rate']==10.0].index
idx_10=idx[:2650000]
del_10=new_data.drop(idx_10)

idx=new_data[new_data['rate']==9.0].index
idx_10=idx[:140000]
del_10=del_10.drop(idx_10)

idx=new_data[new_data['rate']==8.0].index
idx_10=idx[:110000]
del_10=del_10.drop(idx_10)

idx=new_data[new_data['rate']==1.0].index
idx_10=idx[:10000]
del_10=del_10.drop(idx_10)

del_10['rate'].value_counts()


In [ ]:
del_10.to_csv("data_0_all_finish_del.csv",index=False,encoding='utf-8-sig')

In [3]:
new_data=pd.read_csv("data_0_all_finish_del.csv")
new_data

,comment,rate
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6.0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.0
...,...,...
3571030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.0
3571031,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",10.0
3571032,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
3571033,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.0


3000 6000 vocab size == 118798

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
X=pd.read_csv("data_0_all_finish_del.csv")
X['comment']=X['comment'].map(eval)
x_train=np.array(X['comment'])
x=[]
for i in x_train:
    x.append(np.asarray(i).astype('float32'))
x=np.asarray(x)
y=np.asarray(X['rate'])
y=y.reshape(-1,1)

x.shape,y.shape

In [ ]:
model = Sequential()
model.add(Embedding(118798,1000))
model.add(LSTM(128,input_shape=(30,1)))
model.add(Dense(1, activation='relu'))

In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())



In [ ]:
model.compile(optimizer='Adam', loss='mse', metrics=["mae"])
history = model.fit(x, y, epochs=20, callbacks=[es, mc], batch_size=16000, validation_split=0.2,verbose=1)

In [ ]:
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(x, y)

In [ ]:
def sentiment_predict(new_sentence):
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩 위에서 fit 해야함
    pad_new = pad_sequences(encoded, maxlen = max_len)
    score = float(loaded_model.predict(pad_new))
    return score